In [6]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords


In [7]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/cristian/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [10]:
stop_words=list(set(stopwords.words("english")))
print(stop_words[:3])

["didn't", 'd', "shouldn't"]


In [11]:
df = pd.read_excel("../Data/final_adult_content.xlsx")
print("Shape : ",df.shape)
df.head(2)

Shape :  (850, 2)


,Description,Category
0,Washington Correspondent New York Magazine Wri...,Non_Adult
1,Host of NPR s Pop Culture Happy Hour Author NY...,Non_Adult


In [12]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer

token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(lowercase=True,stop_words='english',ngram_range = (1,1),tokenizer = token.tokenize)
text_counts= cv.fit_transform(df['Description'])

/media/cristian/Extreme/Investigacion/Hackathones/2023-Eth/ethe_env/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [13]:
text_counts

<850x3639 sparse matrix of type '<class 'numpy.int64'>'
	with 10838 stored elements in Compressed Sparse Row format>

In [14]:
print(type(text_counts))
print(text_counts.shape)
#print(text_counts.toarray())
print(text_counts.toarray())
#print(cv.vocabulary_)

<class 'scipy.sparse._csr.csr_matrix'>
(850, 3639)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(text_counts, df['Category'], test_size=0.3, random_state=1)

In [17]:
from sklearn.naive_bayes import MultinomialNB
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Generation Using Multinomial Naive Bayes
clf = MultinomialNB().fit(X_train, y_train)
predicted= clf.predict(X_test)
print("MultinomialNB Accuracy:",metrics.accuracy_score(y_test, predicted))

MultinomialNB Accuracy: 0.9607843137254902


In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf=TfidfVectorizer()
text_tf= tf.fit_transform(df['Description'])

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    text_tf, df['Category'], test_size=0.3, random_state=123)

In [20]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
# Model Generation Using Multinomial Naive Bayes
clf = MultinomialNB().fit(X_train, y_train)
predicted= clf.predict(X_test)
print("MultinomialNB Accuracy:",metrics.accuracy_score(y_test, predicted))

MultinomialNB Accuracy: 0.9725490196078431


In [30]:
def predict(sentence):
    preprocessed = tf.transform([sentence])
    return clf.predict(preprocessed)[0]
    

In [31]:
predict("Hello my friend, how are you?")

'Non_Adult'

In [34]:
predict("where can i buy illegal weapons")

'Adult'

In [37]:
import pickle

pickle.dump(tf, open("../Results/preprocess.pkl", 'wb'))
pickle.dump(clf, open("../Results/model.pkl", 'wb'))

In [38]:
tf_model = pickle.load(open("../Results/preprocess.pkl", 'rb'))

In [39]:
clf_model = pickle.load(open("../Results/model.pkl", 'rb'))

In [40]:
def predict(sentence):
    preprocessed = tf_model.transform([sentence])
    return clf_model.predict(preprocessed)[0]
    

In [41]:
predict("where can i buy illegal weapons")

'Adult'